In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use all onehot-keepfirst nominal encoding for coupon type: Coffee House

In [2]:
coupon_type = "CoffeeHouse"

In [3]:
encoding_type = 'onehot-keepfirst' ## use this string to access input file and name output files

os.makedirs(os.path.join('model', encoding_type), exist_ok=True)

In [4]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv(f'data/{coupon_type}_data_{encoding_type}.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 49.86% Yes
After SMOTE upsampling
	Num training samples: 3,200 (50.62% yes)
	Num testing samples:  800 (47.50% yes)


## Testing RandomForestClassifier with Grid Search Cross-validation


## RandomForestClassifier Parameters

The main parameters to adjust when using these methods:
* n_estimators
* max_features (sqrt is generally best default for classification. I have 55 features, so try values around 8) 

Make sure to include params:
* max_depth=None
* min_samples_split=2 
* oob_score=True (with bootstrapping)

In [5]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
#                 'param_min_samples_leaf', 
                'param_max_depth', 'param_max_leaf_nodes',
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [6]:
param_grid={'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            'min_samples_split': [2, 3],
            'n_estimators': [100, 150, 200],
#             'min_samples_leaf': [1, 2, 3, 4, 5, 10],
            'max_depth': [None, 20, 25, 30],
            'max_leaf_nodes': [None, 100, 200]
           }

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [None, 20, 25, 30],
                         'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15],
                         'max_leaf_nodes': [None, 100, 200],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_depth=30, max_features=5, min_samples_split=3,
                       n_estimators=200, oob_score=True, random_state=42)

In [7]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.7715625
ROC AUC: TEST 0.854073



In [8]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,param_max_depth,param_max_leaf_nodes,mean_test_score,std_test_score,mean_train_score,std_train_score
599,1,5,3,200,30,None,0.84717436,0.01272554,0.99992852,0.00001896
5,2,5,3,200,None,None,0.84681200,0.01332239,0.99993023,0.00002286
598,3,5,3,150,30,None,0.84657661,0.01303971,0.99991667,0.00002690
256,4,8,3,150,20,None,0.84640960,0.01278450,0.99988432,0.00005668
4,5,5,3,150,None,None,0.84639299,0.01363251,0.99991826,0.00002928
255,6,8,3,100,20,None,0.84634123,0.01341580,0.99986527,0.00005732
203,7,5,3,200,20,None,0.84602477,0.01506020,0.99981925,0.00004543
257,8,8,3,200,20,None,0.84600133,0.01311784,0.99988566,0.00004767
635,9,7,3,200,30,None,0.84569659,0.01500373,0.99994207,0.00001825
3,10,5,3,100,None,None,0.84568292,0.01246001,0.99989354,0.00003031


In [9]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
91,CoffeeHouse_never,0.06632913
87,CoffeeHouse_1~3,0.02611906
25,expiration_1d,0.01919985
26,expiration_2h,0.01757860
88,CoffeeHouse_4~8,0.01707772
20,time_10AM,0.01658612
90,CoffeeHouse_less1,0.01587495
110,minsToCouponDest_5-14,0.01578565
86,Bar_never,0.01513353
93,CarryAway_1~3,0.01501666


In [10]:
import pickle

with open(f'model/{encoding_type}/{coupon_type}_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open(f'model/{encoding_type}/{coupon_type}_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)

In [11]:
## save train/test info to file
Xy_tt = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

for k in Xy_tt.keys():
    fname = f'data/{coupon_type}_{k}_{encoding_type}.csv'
    print(f'Saving file: {fname}')
    Xy_tt[k].to_csv(fname, index=False)

Saving file: data/CoffeeHouse_X_train_onehot-keepfirst.csv
Saving file: data/CoffeeHouse_X_test_onehot-keepfirst.csv
Saving file: data/CoffeeHouse_y_train_onehot-keepfirst.csv
Saving file: data/CoffeeHouse_y_test_onehot-keepfirst.csv
